In [1]:
# Run locally & on the cloud
import pandas as pd
import numpy as np
import pickle
from sklearn.ensemble import RandomForestRegressor

In [2]:
new_test_data = pd.DataFrame({
    'HallwayType': ['terraced', 'corridor'],
    'TimeToSubway': ['0-5min', '5min~10min'],
    'SubwayStation': ['Kyungbuk_uni_hospital', 'Bangoge'],
    'N_FacilitiesNearBy(ETC)': [1.0, 2.0],
    'N_SchoolNearBy(University)': [3.0, 4.0],
    'N_Parkinglot(Basement)': [1270.0, 1174.0],
    'YearBuilt': [1986, 2008],
    'N_FacilitiesInApt': [10, 3],
    'Size(sqf)': [1761, 1448],
    'SalePrice': [138053, 89380],
    'N_FacilitiesNearBy(PublicOffice)': [6.0, 1.0],
    'Size(SqMetre)': [12345, 321]
})

new_test_data

,HallwayType,TimeToSubway,SubwayStation,N_FacilitiesNearBy(ETC),N_SchoolNearBy(University),N_Parkinglot(Basement),YearBuilt,N_FacilitiesInApt,Size(sqf),SalePrice,N_FacilitiesNearBy(PublicOffice),Size(SqMetre)
0,terraced,0-5min,Kyungbuk_uni_hospital,1.0,3.0,1270.0,1986,10,1761,138053,6.0,12345
1,corridor,5min~10min,Bangoge,2.0,4.0,1174.0,2008,3,1448,89380,1.0,321


In [3]:
#import google cloud library
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import aiplatform

In [4]:
## set up authentication using services account 
import os
# Authenticate using service account
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "sa-development.json"
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../vertex-ai-ml/dti-ds-31329ac0651d.json"

project_id = 'dti-ds'
dataset_id = 'rangga_dataset_010'
table_id = 'x_test_daegu'
region = 'us-central1'
bucket_name = 'rangga_gcs_010'
blob_name = 'data/x_test_daegu.csv'

model_name = 'rangga_daeguapart_model.pkl'

In [5]:
# Retrieve the model from Google Cloud Storage
try : 
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name) # Add bucket name
    blob_model = bucket.blob(f'model/{model_name}')
    blob_model.download_to_filename(f'model/{model_name}')

    print ("Read model succeeded")
except:
    raise TypeError("An exception occurred")

Read model succeeded


In [6]:
loaded_model = pickle.load(open(f'model/{model_name}', 'rb'))

prediction = loaded_model.predict(new_test_data)
prediction

array([443118.27858154, 240646.43777065])

In [7]:
#load data from BQ
from google.cloud import bigquery


client = bigquery.Client(project=project_id)

# query 
query_job = client.query(f"""select * from {dataset_id}.{table_id}""")
df_test = query_job.result().to_dataframe()


df_test.rename(columns={
    'N_FacilitiesNearBy_ETC_': 'N_FacilitiesNearBy(ETC)',
    'N_FacilitiesNearBy_PublicOffice_': 'N_FacilitiesNearBy(PublicOffice)',
    'N_SchoolNearBy_University_': 'N_SchoolNearBy(University)',
    'N_Parkinglot_Basement_': 'N_Parkinglot(Basement)',
    'Size_SqMetre_': 'Size(SqMetre)'}, 
inplace=True)

df_test

/home/ranggaakhli/miniconda3/envs/daegu_env/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,HallwayType,TimeToSubway,SubwayStation,N_FacilitiesNearBy(ETC),N_FacilitiesNearBy(PublicOffice),N_SchoolNearBy(University),N_Parkinglot(Basement),YearBuilt,N_FacilitiesInApt,Size(SqMetre)
0,corridor,0-5min,Kyungbuk_uni_hospital,0.0,3.0,2.0,0.0,1978,3,123.282334
1,corridor,0-5min,Kyungbuk_uni_hospital,0.0,3.0,2.0,0.0,1978,3,83.891445
2,corridor,0-5min,Kyungbuk_uni_hospital,0.0,3.0,2.0,0.0,1978,3,83.891445
3,corridor,0-5min,Kyungbuk_uni_hospital,0.0,3.0,2.0,0.0,1978,3,83.891445
4,corridor,0-5min,Kyungbuk_uni_hospital,0.0,3.0,2.0,0.0,1978,3,83.891445
...,...,...,...,...,...,...,...,...,...,...
635,mixed,15min~20min,Myung-duk,1.0,5.0,4.0,79.0,1992,3,59.829558
636,mixed,15min~20min,Myung-duk,1.0,5.0,4.0,79.0,1992,3,134.523602
637,mixed,15min~20min,Myung-duk,1.0,5.0,4.0,79.0,1992,3,84.913379
638,mixed,15min~20min,Myung-duk,1.0,5.0,4.0,79.0,1992,3,59.829558


In [8]:
y_pred = loaded_model.predict(df_test)
y_pred_df = pd.DataFrame(y_pred, columns=['SalePrice'])

result_df = pd.concat([df_test.reset_index(drop=True), y_pred_df.reset_index(drop=True)], axis=1)
result_df

,HallwayType,TimeToSubway,SubwayStation,N_FacilitiesNearBy(ETC),N_FacilitiesNearBy(PublicOffice),N_SchoolNearBy(University),N_Parkinglot(Basement),YearBuilt,N_FacilitiesInApt,Size(SqMetre),SalePrice
0,corridor,0-5min,Kyungbuk_uni_hospital,0.0,3.0,2.0,0.0,1978,3,123.282334,171776.420299
1,corridor,0-5min,Kyungbuk_uni_hospital,0.0,3.0,2.0,0.0,1978,3,83.891445,170193.458791
2,corridor,0-5min,Kyungbuk_uni_hospital,0.0,3.0,2.0,0.0,1978,3,83.891445,170193.458791
3,corridor,0-5min,Kyungbuk_uni_hospital,0.0,3.0,2.0,0.0,1978,3,83.891445,170193.458791
4,corridor,0-5min,Kyungbuk_uni_hospital,0.0,3.0,2.0,0.0,1978,3,83.891445,170193.458791
...,...,...,...,...,...,...,...,...,...,...,...
635,mixed,15min~20min,Myung-duk,1.0,5.0,4.0,79.0,1992,3,59.829558,114914.739777
636,mixed,15min~20min,Myung-duk,1.0,5.0,4.0,79.0,1992,3,134.523602,180365.838482
637,mixed,15min~20min,Myung-duk,1.0,5.0,4.0,79.0,1992,3,84.913379,149953.734883
638,mixed,15min~20min,Myung-duk,1.0,5.0,4.0,79.0,1992,3,59.829558,114914.739777


In [9]:
result_df.rename(columns={
    'N_FacilitiesNearBy(ETC)': 'N_FacilitiesNearBy_ETC_', 
    'N_FacilitiesNearBy(PublicOffice)': 'N_FacilitiesNearBy_PublicOffice_',
    'N_SchoolNearBy(University)': 'N_SchoolNearBy_University_',
    'N_Parkinglot(Basement)': 'N_Parkinglot_Basement_',
    'Size(SqMetre)': 'Size_SqMetre_'
}, inplace=True)

In [10]:
from google.cloud.exceptions import NotFound

dest_table_id = 'saleprice_prediction'
table_full_id = f'{project_id}.{dataset_id}.{dest_table_id}'

schema = [
    bigquery.SchemaField('HallwayType', 'STRING'),
    bigquery.SchemaField('TimeToSubway', 'STRING'),
    bigquery.SchemaField('SubwayStation', 'STRING'),
    bigquery.SchemaField('N_FacilitiesNearBy_ETC_', 'FLOAT'),
    bigquery.SchemaField('N_FacilitiesNearBy_PublicOffice_', 'FLOAT'),
    bigquery.SchemaField('N_SchoolNearBy_University_', 'FLOAT'),
    bigquery.SchemaField('N_Parkinglot_Basement_', 'FLOAT'),
    bigquery.SchemaField('YearBuilt', 'INTEGER'),
    bigquery.SchemaField('N_FacilitiesInApt', 'INTEGER'),
    bigquery.SchemaField('Size_SqMetre_', 'FLOAT'),
    bigquery.SchemaField('SalePrice', 'FLOAT')

]

table_ref = client.dataset(dataset_id).table(dest_table_id)

# Check if the table exists
try:
    client.get_table(table_ref)
    print(f'Table {table_full_id} already exists.')
except NotFound:
    # Create the table if it does not exist
    table = bigquery.Table(table_ref, schema=schema)
    table = client.create_table(table)
    print(f'Table {table_full_id} created.')

job_config = bigquery.LoadJobConfig(
    schema=schema,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE
)

load_job = client.load_table_from_dataframe(result_df, table_ref, job_config=job_config)
load_job.result()

print(f'Loaded {load_job.output_rows} rows into {table_full_id}')

Table dti-ds.rangga_dataset_010.saleprice_prediction already exists.
Loaded 640 rows into dti-ds.rangga_dataset_010.saleprice_prediction
